In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cryptography.fernet import Fernet
import re
import datetime as dt

In [62]:
dfPrinc = pd.read_csv('../../dados/dfPrincipal.csv')

In [63]:
dfPrinc.head()

,Unnamed: 0,SEXO,DT_NASCIMENTO,FORMA_INGRESSO,FORMA_EVASAO,PERIODO_INGRESSO,DT_EVASAO,PERIODO_EVASAO,PER_EVASAO_FORMAT,ANO_EVASAO,PER_INGRESSO_FORMAT,ANO_INGRESSO
0,0,M,1982-02-09,VE - Vestibular,CON - Curso concluído,2,2009-09-14,2.0,2009.2,2009.0,2002.2,2002
1,1,M,1977-05-09,VE - Vestibular,ABA - Abandono do curso,1,2005-03-12,2.0,2005.2,2005.0,2004.1,2004
2,2,M,1985-03-05,VE - Vestibular,Não identificada (vide pasta do aluno),1,2003-01-06,1.0,2003.1,2003.0,2003.1,2003
3,3,M,1988-07-18,SISU Ampla Concorrencia,CON - Curso concluído,2,2018-03-23,2.0,2017.2,2017.0,2013.2,2013
4,4,M,2002-07-02,"SISU Escola Pública, Indep. de Renda : Preto e...",Sem evasão,1,NaN,NaN,nan.,NaN,2020.1,2020


### Analise Dados Alunos



--- 

### Grupos de Alunos

| /                           |Todos os Alunos | Antes de 2013 | De 2013 até 2020 | De 2020 até 2021 
| -----------                 | -----------    |  -----------  | -----------      | -----------     | 
Análise da % de conclusão     | x              |  -----------  | -----------      | -----------     | 
Análise da % de genero        | x              |  -----------  | -----------      | -----------     | 
Análise do perfil de idade    | -----------    |  -----------  | -----------      | -----------     | 
Análise da distancia          | -----------    |  -----------  | -----------      | -----------     | 
Análise do rendimento escolar | -----------    |  -----------  | -----------      | -----------     | 
Análise da % de conclusão     | -----------    |  -----------  | -----------      | -----------     | 







## Análise da % de conclusão(Todos os Alunos)

In [8]:
evasoesDF = dfPrinc.groupby(['ANO_EVASAO','FORMA_EVASAO'], dropna=False).size().reset_index(name='ALUNOS')

In [9]:
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('ABA - Abandono do curso', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('APO - Aluno Especial - Disciplina Isolada', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('APO -Conclusão a Mobilidade Acadêmica Externa - ANDIFES', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('APO -Conclusão da Mobilidade Acadêmica Externa - IAE/IESCE', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('Desistência SiSU', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('JUB - Jubilamento', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('CAN - Cancelamento Geral do curso', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('CON - Curso concluído', 'Concluído') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('Sem evasão', 'Cursando') 

In [10]:
evasoesDF.loc[evasoesDF['FORMA_EVASAO'] == 'Não identificada (vide pasta do aluno)' , 'FORMA_EVASAO'] = 'Evasão'

In [11]:
evasoesDF = evasoesDF.groupby(['FORMA_EVASAO'] , as_index=False)['ALUNOS'].sum() 

In [12]:
# Mostra porcentagem de alunos
evasoesDF['percent'] =  (100. * evasoesDF['ALUNOS'] / evasoesDF['ALUNOS'].sum()).round(0)


In [13]:
evasoesDF

,FORMA_EVASAO,ALUNOS,percent
0,Concluído,335,27.0
1,Cursando,418,33.0
2,Evasão,507,40.0


---

## Análise da % de gênero

In [14]:
generoDF = dfPrinc.groupby(['SEXO'], dropna=False).size().reset_index(name='ALUNOS')

In [16]:
generoDF['percent'] =  (100. * generoDF['ALUNOS'] / generoDF['ALUNOS'].sum()).round(0)

In [17]:
generoDF

,SEXO,ALUNOS,percent
0,F,238,19.0
1,M,1022,81.0


## Análise do perfil de idade


In [50]:
dfPrinc['DT_NASCIMENTO'] = pd.to_datetime(dfPrinc['DT_NASCIMENTO'])


In [66]:
#colunas = ['SEXO', 'FORMA_INGRESSO','FORMA_EVASAO','PERIODO_INGRESSO','DT_EVASAO','PERIODO_EVASAO','PER_EVASAO_FORMAT','ANO_EVASAO',
#           'PER_INGRESSO_FORMAT']
colunas = ['DT_NASCIMENTO','ANO_INGRESSO']
idadeDF = dfPrinc.filter(colunas)


In [47]:
arrayDatas = dfPrinc['DT_NASCIMENTO'].array
arrayIngresso = dfPrinc['ANO_INGRESSO'].array
arrayIdadeFormatado = []
def getIdadeIngresso(DT_NASCIMENTO, ANO_INGRESSO):
    return arrayIdadeFormatado.append(ANO_INGRESSO - DT_NASCIMENTO.year)
    

In [85]:
for dt,ing in zip(arrayDatas,arrayIngresso):
    getIdadeIngresso(dt, ing)

In [100]:
idadeDF['IDADE'] = arrayIdadeFormatado
idadeDF = idadeDF.fillna(0)
idadeDF['IDADE'] = idadeDF['IDADE'].astype(int)

In [102]:
idadeDF

,DT_NASCIMENTO,ANO_INGRESSO,IDADE
0,1982-02-09,2002,20
1,1977-05-09,2004,27
2,1985-03-05,2003,18
3,1988-07-18,2013,25
4,2002-07-02,2020,18
...,...,...,...
1255,2000-07-22,2020,20
1256,1996-02-07,2014,18
1257,1992-03-29,2009,17
1258,1991-02-17,2009,18


#### Fazendo Group By

In [106]:
idadeDF = idadeDF.groupby(['IDADE'], dropna=True).size().reset_index(name='ALUNOS')

In [56]:
idadeDF['ANO_INGRESSO'] = dfPrinc['ANO_INGRESSO']

In [109]:
idadeDF

,IDADE,ALUNOS,PERCENT
0,0,1,0.08
1,17,20,1.62
2,18,249,20.11
3,19,290,23.42
4,20,166,13.41
5,21,103,8.32
6,22,58,4.68
7,23,56,4.52
8,24,63,5.09
9,25,43,3.47


In [ ]:
idadeDF.to_csv ('../../dados/idadeDF.csv', index = False, header=True)
print(idadeDF)

In [108]:
idadeDF['PERCENT'] =  (100. * idadeDF['ALUNOS'] / idadeDF['ALUNOS'].sum()).round(2)